In [1]:
from pynq import Overlay
Overlay("base.bit").download()

In [5]:
from time import sleep
import math
from pynq.board import Button
from pynq.iop import Arduino_IO
from pynq.iop import ARDUINO
from pynq.iop import PMODA
from pynq.iop import PMOD_GROVE_G4
from pynq.iop import ARDUINO_GROVE_I2C
from pynq.iop import Grove_IMU
sense = Grove_IMU(PMODA, PMOD_GROVE_G4)
led = Arduino_IO(ARDUINO,13,'out')
led1 = Arduino_IO(ARDUINO,12,'out')
led2 = Arduino_IO(ARDUINO,11,'out')
print('start')
btns = [Button(index) for index in range(4)]
led.write(0)
led1.write(0)
led2.write(0)

start


In [6]:
def getAngleDeg(x,y,mod):
    h=math.sqrt(x*x + y*y)
    rawAng=math.asin(y/h) / (2*math.pi) *360
    if (x>=0 and y>=0):
        adjAng = rawAng
    elif (x<0 and y>=0):
        adjAng = 180-rawAng
    elif (x>=0 and y<0):
        adjAng = 360+rawAng
    elif (x<0 and y<0):
        adjAng = 180-rawAng
    else:
        adjAng = rawAng
        print('error in angle calcs')
        return 0;
    finalAng= adjAng + mod;
    return finalAng
    
def get_tilt_heading(data,altdata):
    [ax, ay, _] = altdata[0:3]
    [mx, my, mz] = data[0:3]
    try:
        pitch = math.asin(-ax)
        roll = math.asin(ay / math.cos(pitch))
    except ZeroDivisionError:
        raise RuntimeError("Value out of range or device not connected")

    xh = mx * math.cos(pitch) + mz * math.sin(pitch)
    yh = mx * math.sin(roll) * math.sin(pitch) + \
    my * math.cos(roll) - mz * math.sin(roll) * math.cos(pitch)
    _ = -mx * math.cos(roll) * math.sin(pitch) + \
    my * math.sin(roll) + mz * math.cos(roll) * math.cos(pitch)
    tilt_heading = 180 * math.atan2(yh, xh) / math.pi
    if yh < 0:
        tilt_heading += 360
    return float("{0:.2f}".format(tilt_heading))

In [10]:
print('start')
sense.reset()
data=sense.get_compass()
heading=sense.get_heading()
altdata=sense.get_accl()
print(data)
while (btns[3].read()==0):
    sense.reset()
    data=sense.get_compass()
    heading=get_tilt_heading(data, altdata)
    print(data)
    print(getAngleDeg(data[0],data[1],0))
    print(heading)
    if (heading < 5 or heading >355):
        led.write(1)
        led1.write(0)
        led2.write(0)
    elif (heading < 30 and heading > 5):
        led1.write(0)
        led.write(0)
        led2.write(1)
    elif (heading < 355 and heading > 330):
        led2.write(0)
        led.write(0)
        led1.write(1)
    else:
        led.write(0)
    #print(altitude(data[0],data[1],data[2]))
    sleep(2)
print('done')
led.write(0)
led1.write(0)
led2.write(0)

start
[15.53, 4.39, 12.3]
[16.11, 4.39, 12.3]
15.24308051024971
14.28
[15.53, 4.1, 12.6]
14.788953532296269
13.75
[14.36, 3.22, 12.89]
12.638601963567153
11.41
[14.65, 3.22, 12.01]
12.396226779799878
11.27
[14.65, 2.34, 12.6]
9.075022859918892
7.77
[12.89, 1.17, 12.6]
5.186413269914536
3.57
[12.89, 0.59, 12.01]
2.6207084529265345
1.0
[12.01, -0.88, 11.13]
355.8092970069573
354.01
[14.06, 0.59, 11.13]
2.402893939521624
1.02
[13.18, 0.88, 11.43]
3.8198453831189734
2.34
[11.72, -0.88, 11.13]
355.7059853289635
353.86
[13.77, 3.22, 11.72]
13.1616417003128
12.02
[14.36, 0.88, 10.84]
3.506776577641219
2.21
[13.77, 0.29, 10.55]
1.2064865760115953
359.83
[14.65, 1.46, 11.43]
5.691231223018669
4.41
[14.36, 1.76, 11.43]
6.987475304256112
5.72
[14.65, 1.76, 11.43]
6.850484200342186
5.6
[13.77, 0.29, 11.13]
1.2064865760115953
359.76
[14.06, 1.17, 11.13]
4.7568966463741384
3.43
[13.77, 1.17, 11.13]
4.85660393394318
3.51
[14.06, 0.88, 11.13]
3.5814083901729887
2.22
done
